#  Introdução ao Python
## Professor: Luiz Ferreira

## Módulo 4

### Python para Dados

_____________

## O que é DataScience?

Imaginemos que fomos contratados por uma empresa, chamada DataSciencester! Chegamos na empresa para nosso primeiro dia de trabalho.

No primeiro dia, temos o vice-presidente de Rede da empresa, que está louco pelo seu trabalho, uma vez que sempre teve dúvidas sobre ses funcionários e não tnha ninguém para perguntar.

Particularmente, ele quer que a gente encontre as amizades entre os funcionários de nossa empresa, para que possa ver os laços que cada um criou lá dentro. Com isso ele lhe fornece uma parte de toda a rede para a consulta.

Os dados se apresentam na forma de `dict` (conforme vimos em muitas outras aulas), que consiste em uma informação chamada id e uma outra chamada name.

Vejamos:

In [ ]:
users = [
    {"id":0,"name":"Hero"},
    {"id":1,"name":"Dunn"},
    {"id":2,"name":"Sue"},
    {"id":3,"name":"Chi"},
    {"id":4,"name":"Thor"},
    {"id":5,"name":"Clive"},
    {"id":6,"name":"Hicks"},
    {"id":7,"name":"Devin"},
    {"id":8,"name":"Kate"},
    {"id":9,"name":"Klein"}
]

O que podemos perceber desses dados?

Eita, temos mais algumas informações. É uma lista de pares de ID's chamada friendships, vejamos:

In [ ]:
friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4), (4,5), (5,6), (5,7), (6,8), (7,8), (8,9)]

Como podemos entender essas informações?

Vamos criar a lista de amigos de cada pessoa? Para isso temos de acrescentar um campo ao nosso `dict`. Portanto:

In [ ]:
for user in users:
    user["friends"]=[]

Pronto, com isso, podemos agora inserir as informações dentro do novo campo.

In [ ]:
for i, j in friendships:
    users[i]["friends"].append(users[j])
    users[j]["friends"].append(users[i])

Agora que temos uma lista de pessoas e suas amizades, podemos calcular a quantidade média de amigos dentro de nossa empresa? 

Devemos dividir o número de amizades pelo número de pessoas.

In [ ]:
def number_of_friends(user):
    """Quantos amigos o usuário tem?"""
    return len(user["friends"])

total_connections = sum(number_of_friends(user) for user in users)

"""Então dividimos o número de amizades pelo número de usuários"""

num_users = len(users)
avg_connections = total_connections / num_users

Pronto, a variável `avg_connections` agora poderá nos informar a média de amigos por user.

Imaginemos então como ficaria nossa lista de amizades em formato de grafo? Quais insights podemos tirar dessa imagem?

______________


### Não paramos por ai

O vice-diretor de Fraternidade assustou com a baixa relação entre a equipe de trabalho, ele acredita que para termos um ambiente propício ao crescimento, devemos trabalhar de forma conjunta, por isso ele quer estimular a amizade dentro de nossa empresa. Inspirado no Facebook e Instagram, ele nos pediu para desenvolver sugestões de "Cientistas de Dados Que Você Talvez Conheça".

Daí temos outro insight, é mais fácil uma amizade surgir com base nas amizades que já existem, ou seja, amigos de amigos.

Pensando nisso, desenvolvemos uma função para recuperar os amigos de amigos de todos os cientistas.

In [ ]:
def friends_of_friend_ids_bad(user):
    #usaremos foaf como abreviatura
    return [foaf["id"]
            for friend in user["friends"]   #para cada amigo de usuário
            for foaf in friend["friends"]]  #pega cada _their_friends

print(friends_of_friend_ids_bad(users[0])) # com isso temos uma lista dos amigos dos amigos de Hero

Compilando esse comando, percebemos que alguns amigos repetem na lista, pois algumas pessoas podem ser acessadas através de amizades diferentes e essa informação pode ser útil futuramente. Então talvez devêssemos produzir uma contagem de amigos em comum, e uma função de ajuda para excluir as pessoas que já são conhecidas do usuário:

In [ ]:
#Vamos melhorar nossa função
from collections import Counter

def not_the_same(user, other_user):
    #dois usuários não são os mesmos se possuem ids diferentes
    return user["id"] != other_user["id"]

def not_friends(user, other_user):
    #other_user não é um amigo se não está em user["friends"]
    # isso é, se é not_the_same com todas as pessoas em user["friends"]
    return all(not_the_same(friend, other_user)
               for friend in user["friends"])
    
def friends_of_friend_ids(user):
    return Counter(foaf["id"]
                   for friend in user["friends"]    #para cada um dos meus amigos
                   for foaf in friend["friends"]    #que contam *their* amigos
                   if not_the_same(user,foaf)       #que não seja eu
                   and not_friends(user,foaf))      #e que não são meus amigos

Se compilarmos, teremos:

In [ ]:
print(friends_of_friend_ids(users[3]))

Teremos com resposta: Counter({0:2, 5:1})

Isso significa que o Chi (users[3]) tem dois amigos em comum com o Hero (users[0]) e um amigo com o Clive (users[5]).

Mas ainda não acabamos com os problemas que o chefe pediu.

Como um bom cientista de dados, sabemos que podemos ter amizades fruto de interesses em comum de usuário (esse conhecimento é o aspecto de competência significativa falada anteriormente). Depois de uma entrevista, conseguimos essas informações, uma lista de pares com (user_id, interest):

In [ ]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"), (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"), (1, "PostGres"), (2, "Python"), (2, "scikit-learn"),
    (2, "scipy"), (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"), (3, "statistics"), (3, "regression"),
    (3, "probability"), (4, "machine learning"), (4, "regression"), (4, "decision trees"), (4, "libsvm"), (5, "Python"),
    (5, "R"), (5, "Java"), (5, "C++"), (5, "Haskell"), (5, "programing languages"), (6, "theory"), (7, "machine learning"), 
    (7, "scikit-learn"), (7, "Mahout"), (7, "neural networks"), (8, "neural networks"), (8, "deep learning"), (8, "Big Data"), 
    (8, "artificial intelligence"), (9, "Hadoop"), (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

Por alto podemos ver que Thor (id 4) não possui amigos em comum com Devin (id 7), mas compartilham do mesmo interesse em aprendizado de máquina.

Podemos construir uma função que encontre usuários com o mesmo interesse:

In [ ]:
def data_scientists_who_like(target_interest):
    return [user_id 
           for user_id, user_interest in interests
           if user_interest == target_interest]

Nossa função funciona, porém , temos um limitante, cada vez que executarmos ele terá de percorrer toda a lista, isso pode levar um tempo caso nossa base seja grande. Podemos melhorar isso. 

Vamos construir duas listas, uma como um índice de interesses para usuários e uma de usuários para interesses.

In [ ]:
from collections import defaultdict

user_ids_by_interests = defaultdict(list)
for user_id , interest in interests:
    user_ids_by_interests[interest].append(user_id)
    
interests_by_user_id = defaultdict(list)
for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)

Pronto, agora sim, ficou mais fácil descobrir quem possui os maiores interesses em comum com um certo usuário:

    - Itera sobre os interesses do usuário
    - Para cada interesse, itera sobre os outros usuários com aquele interesse
    - Mantém a contagem de quantas vezes cemos cada outro usuário

In [ ]:
def most_common_interests_with(user):
    return Counter(interested_user_id
                  for interest in interests_by_user_id[user["id"]]
                  for interested_user_id in user_ids_by_interests[interest]
                  if interested_user_id != user["id"])

Com o print dessa função para o usuário 3, teremos: 

Counter({0: 2, 5: 1})
Counter({5: 2, 2: 1, 4: 1})

Que significa quantos interesses em comum o usuário possui.

Poderíamos usar esse exemplo para construir um recurso mais rico de "Cientistas de Dados Que Você Deveria Conhecer" baseado em uma combinação de amigos e interesses em comum.